### Import necessary names
Always run the cell below after kernel restart.

In [ ]:
from pybeam.datamodels import nodes_from_csv, elements_from_csv
from pybeam.assembly import assemble_system_matrices, reindex_dof
from pybeam.simulation import simulate
from pybeam.modal_parameters import get_modal_parameters
from pybeam._utilities import pprint_array
from pybeam.plotting import plot_structure, plot_modeshape, set_style, plot_deformations

%matplotlib qt5
set_style()
import numpy as np

import logging

logging.getLogger("pybeam").setLevel(logging.INFO)

### Initialize system matrices

Assemble the system matrices.

The damping matrix is defined by Rayleigh's damping model. Change coefficients as needed.

In [ ]:
nodes = nodes_from_csv("data/frame/nodes.csv")
elements = elements_from_csv("data/frame/elements.csv", nodes)

stiffness, mass = assemble_system_matrices(elements)
damping = 0.01 * mass + 0.001 * stiffness
# damping = None

elements = reindex_dof(elements)

plot_structure(elements, node_labels=True, element_labels=True)

### Get eigenfrequencies and modeshapes

The cell below will compute eigenfrequencies and the modal matrix.

In [ ]:
eigenfrequencies, modeshapes = get_modal_parameters(stiffness, mass)

print(f"{'Mode number':<11s} | {'Eigenfrequency [Hz]':>19} |{'Eigenfrequency [rad/s]':>23} | {'Min dt':>8}")
for i, omega in enumerate(eigenfrequencies):
    print(f"Mode {i:>6d} |{omega / (2 * 3.1415):>20.4f} |{omega:>23.4f} |{(3.1415) / omega:>9.6f}")

### Plot modeshapes

The cell below will plot the mesh and a modeshape. Control what modeshape to plot with the `mode` argument.

In [ ]:
plot_structure(elements, node_labels=False, element_labels=False)
plot_modeshape(elements, modeshapes, mode=0, scale=1)

### Set simulation parameters

Set parameters for the time-domain simulation.

Parameters you can play around with:

* `dt`: Time step size
* `t_end`: Last time step
* `loads`: This is the load time series. You define loads by indexing into the `loads` variable, which is an array.
  * The index `loads[0, :]` means "0th DOF, all columns (all time steps)"
  * To define the load in the 3rd DOF as a cosine function with a frequency of 10 rad/s and an amplitude of 20, do:

    ```python
    loads[3, :] = 20 * np.cos(10 * t)
    ```

  * To define the load in the 2nd to last DOF as a step function with amplitude 40, starting at time 30, do:

    >```python
    ># First, we get the index of the time step that has value 40
    ># `np.where` returns an 1 x n array, where n is the number of matches.
    ># We use [0][0] to get the 0th row, and 0th column.
    >start_index = np.where(time==30)[0][0]
    ># Now we set the value of the load to 40 for all time steps where time>=30
    ># The [-2, start_index:] index means "second to last row, all columns from start_index to the end"
    >loads[-2, start_index:] = 40
    >```


* `x_0`, `v_0`: Initial displacements and initial velocities. Both are n_dof x 1 vectors.
  * To set the initial displacement in the last DOF to 5, do

    >```python
    ># The [-1] index means "last item".
    >x_0[-1] = 5
    >```

  * You can also set several values at once. If we want to set the initial velocity of DOFs 0, 5 and 8 to 0.1, 0.5 and 1.2, respectively, we can do:

    >```python
    ># Node the nested square brackets - [[...]] instead of [...].
    ># It means that we are indexing into several items along that axis.
    >v_0[[0, 5, 8]] = 0.1, 0.5, 1.2
    ># In this case, there is only one axis, because we are dealing with vectors.
    ># If we were dealing with a 2D array/matrix, we could do
    > # [[0, 5, 8], [0, 1]] to get rows 0, 5 and 8, columns 0 and 1.
    >```

* `beta`: This is an integration parameter in the Newmark algorithm. `beta=1/6` means that we are using linear acceleration, and `beta=1/4` means that we are using constant acceleration between two time steps.
  * Note: If you are using `beta=1/6`, the algorithm is conditionally stable. If you are getting an unstable time response, do one of these three:
    1. Decrease `dt`. This will give you more time steps, which means more computing power is needed, but it will give stable results once it becomes small enough.
    2. Increase the damping. Simply adjusting the damping to your needs is not physical, but numerically, it can give stability.
    3. Use `beta=1/4`. In this case the algorithm is unconditionally stable. However, constant acceleration gives a less precise representation of the physical structure. You can try to mitigate this by decreasing `dt` to get shorter time steps. This will improve our approximation of the physical structure's behaviour, but also be more computationally expensive.

In [ ]:
# Number of DOF
n_dof = stiffness.shape[0]

# Time step size
dt = 0.005

# End time
t_end = 2
# Time steps
time = np.arange(start=0, stop=t_end + dt, step=dt)

# Loading
loads = np.zeros((n_dof, len(time)))
loads[9, :] = 100 * np.cos((eigenfrequencies[0] + eigenfrequencies[1]) / 2 * time)

loads[13, :] = 50 * np.cos(0.8 * eigenfrequencies[2] * time)

# Initial conditions
x_0 = np.zeros((n_dof,))
# x_0[16] = 0.5
v_0 = np.zeros((n_dof,))

# Integration parameter
beta = 1 / 4

### Run simulation

Simulate the time response. This might be a bit slow the first time you run it, but will speed up significantly after that.

In [ ]:
x, v, a = simulate(
    stiffness=stiffness,
    mass=mass,
    damping=damping,
    initial_disp=x_0,
    initial_vel=v_0,
    loads=loads,
    time=time,
    beta=beta,
    gamma=1 / 2,  # Don't change this unless you know what you're doing
)

### Plot the time response

The cell below will plot the time response.

Change the index in `x[-2, :]` to plot the response for other DOF. For example, `x[0, :]` will plot the response for the 0th DOF.

You can also plot the velocity or acceleration response, by replacing `x` with `v` or `a`, or you can plot a load from the `loads` variable.

In [ ]:
import matplotlib.pyplot as plt

ax = plt.gca()

# Adjust the look of the plot
ax.set_xlabel("Time")
ax.set_ylabel("Displacement")
ax.set_xlim(0, time[-1])

ax.plot(time, x[16, :], color="white")
plt.show()